In [1]:
from deepuq.models.models import model_setup_DE
from deepuq.data.data import DataPreparation
from deepuq.train import train
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader

In [2]:
# necessary in order to display matplotlib plots in a notebook environment
# when also running python scripts inline
%matplotlib inline

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_type = 'bnll_loss'
model, lossFn = model_setup_DE(loss_type, DEVICE)

In [4]:
uniform = True
norm = False
verbose = False
val_prop = 0.1
rs_prior = 42
rs_uniform = 42
rs_train_val = 42
BATCH_SIZE = 100
lr = 0.001
n_models = 15
n_epochs = 100
out_dir = '../DeepUQResources/'

Create a dictionary with all the information for all the experiments from the paper.

In [5]:
experiments_df = {
    "0D, input, low": {
        "size_df": 100000,
        "noise": "low",
        "dim": "0D",
        "injection": "input"
    },
    "0D, input, medium": {
        "size_df": 100000,
        "noise": "medium",
        "dim": "0D",
        "injection": "input"
    },
    "0D, input, high": {
        "size_df": 100000,
        "noise": "high",
        "dim": "0D",
        "injection": "input"
    },
    "0D, output, low": {
        "size_df": 100000,
        "noise": "low",
        "dim": "0D",
        "injection": "output"
    },
    "0D, output, medium": {
        "size_df": 100000,
        "noise": "medium",
        "dim": "0D",
        "injection": "output"
    },
    "0D, output, high": {
        "size_df": 100000,
        "noise": "high",
        "dim": "0D",
        "injection": "output"
    },
    "2D, input, low": {
        "size_df": 5000,
        "noise": "low",
        "dim": "2D",
        "injection": "input"
    },
    "2D, input, medium": {
        "size_df": 5000,
        "noise": "medium",
        "dim": "2D",
        "injection": "input"
    },
    "2D, input, high": {
        "size_df": 5000,
        "noise": "high",
        "dim": "2D",
        "injection": "input"
    },
    "2D, output, low": {
        "size_df": 5000,
        "noise": "low",
        "dim": "2D",
        "injection": "output"
    },
    "2D, output, medium": {
        "size_df": 5000,
        "noise": "medium",
        "dim": "2D",
        "injection": "output"
    },
    "2D, output, high": {
        "size_df": 5000,
        "noise": "high",
        "dim": "2D",
        "injection": "output"
    },
}

In [7]:
#counter = 0
for experiment_name, params in experiments_df.items():
    print(f"Experiment: {experiment_name}")
    for key, value in params.items():
        print(f"  {key}: {value}")
    data = DataPreparation()
    model_inputs, model_outputs = data.generate_df(
                params["size_df"], params["noise"],
                params["dim"], params["injection"], uniform, verbose,
                rs_prior=rs_prior, rs_uniform=rs_uniform)
    model_inputs, model_outputs, norm_params = data.normalize(
        model_inputs, model_outputs, norm
    )
    x_train, x_val, y_train, y_val = data.train_val_split(
        model_inputs,
        model_outputs,
        val_proportion=val_prop,
        random_state=rs_train_val,
    )
    '''
    print(np.shape(x_train), np.shape(y_train))
    if params["dim"] == "2D":
        plt.clf()
        plt.imshow(x_train[0])
        plt.annotate(str(y_train[0]), xy=(0.05,0.8), xycoords='axes fraction', color = 'white', size=20)
        plt.show()
    elif params["dim"] == "0D":
        print(np.shape(x_train[0:100,0]), np.shape(y_train[0:100]))
        plt.scatter(x_train[0:100,0], y_train[0:100])
        plt.show()
    '''
    trainData = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
    trainDataLoader = DataLoader(
        trainData, batch_size=BATCH_SIZE, shuffle=True
    )
    train.train_DE(
            trainDataLoader,
            x_val,
            y_val,
            lr,
            DEVICE,
            'bnll_loss',
            n_models,
            norm_params,
            model_name='DE',
            BETA=0.5,
            EPOCHS=n_epochs,
            path_to_model=out_dir,
            inject_type=params["injection"],
            data_dim=params["dim"],
            noise_level=params["noise"],
            save_all_checkpoints=True,
            save_final_checkpoint=True,
            overwrite_final_checkpoint=False,
            plot=False,
            savefig=False,
            rs_list=[42,43,44,45,46,47,48,49,50,51,52,53,54,55,56],
            set_and_save_rs=True,
        )

Experiment: 0D, input, low
  size_df: 100000
  noise: low
  dim: 0D
  injection: input


/Users/rnevin/Library/Caches/pypoetry/virtualenvs/deepuq-neurips-ws-2024-TMQ0mHE3-py3.10/lib/python3.10/site-packages/deepuq/data/data.py:202: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  df[key] = torch.tensor(value)


starting here 0
model 0
setting and saving the rs
model is Sequential(
  (0): Model(
    (model): Sequential(
      (0): Linear(in_features=3, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): ReLU()
      (4): Linear(in_features=64, out_features=2, bias=True)
    )
  )
  (1): MuVarLayer()
) lossfn <function loss_bnll at 0x135574940>
epoch 0 0.0
new best loss -0.06560062617063522 in epoch 0
epoch 1 0.01
new best loss -0.07883953303098679 in epoch 1
epoch 2 0.02
epoch 3 0.03
epoch 4 0.04
epoch 5 0.05
epoch 6 0.06
epoch 7 0.07
epoch 8 0.08
epoch 9 0.09
epoch 10 0.1
epoch 11 0.11
epoch 12 0.12
epoch 13 0.13
epoch 14 0.14
epoch 15 0.15
epoch 16 0.16
epoch 17 0.17
epoch 18 0.18
epoch 19 0.19
epoch 20 0.2
epoch 21 0.21
epoch 22 0.22
epoch 23 0.23
epoch 24 0.24
epoch 25 0.25
epoch 26 0.26
epoch 27 0.27
epoch 28 0.28
epoch 29 0.29
epoch 30 0.3
epoch 31 0.31
epoch 32 0.32
epoch 33 0.33
epoch 34 0.34
epoch 35 0.35
epoch 36 0.36